# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096-iap17/blob/master/lecture1/Boxes-and-registers.ipynb.)

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [5]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

10000000-element Array{Float64,1}:
 0.845028 
 0.321898 
 0.0471733
 0.0975601
 0.174604 
 0.480109 
 0.681603 
 0.833004 
 0.48757  
 0.975825 
 0.821781 
 0.339295 
 0.682643 
 ⋮        
 0.446681 
 0.804288 
 0.904222 
 0.116732 
 0.310687 
 0.036794 
 0.0386665
 0.629526 
 0.672523 
 0.980441 
 0.474211 
 0.511631 

In [2]:
sum(a)   

5.001419042481687e6

The expected result is 0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

Julia has a `BenchmarkTools.jl` package for easy and accurate benchmarking:

In [3]:
Pkg.add("BenchmarkTools")

INFO: Updating cache of BenchmarkTools...
INFO: Updating cache of LegacyStrings...
INFO: Installing BenchmarkTools v0.0.7
INFO: Installing Blosc v0.2.0
INFO: Installing HDF5 v0.7.3
INFO: Installing JLD v0.6.9
INFO: Installing LegacyStrings v0.2.1
INFO: Building Homebrew


Updated 3 taps (homebrew/core, homebrew/science, staticfloat/juliadeps).
==> New Formulae
adr-tools
afuse
alexjs
allure
angular-cli
ansible-lint
antlr4-cpp-runtime
apple-gcc42
archivemount
asdf
audacious
avfs
awk
awslogs
bandcamp-dl
bartycrouch
bc
bdsup2sub
binaryen
bindfs
btfs
bzip2
c14-cli
chronograf
ck
cmark-gfm
cockroach
conjure-up
corebird
curlftpfs
cython
dcos-cli
diffstat
diffutils
djmount
docker-machine-driver-vultr
druid
ed
encfs
exa
expect
ext2fuse
ext4fuse
file-formula
firebase-cli
flif
flint-checker
fsql
fstrm
fuse-zip
fzy
gcc@6
gcsfuse
gitfs
gitter-cli
gnupg@1.4
gnupg@2.0
goad
goofys
gpatch
gperf
grakn
grep
groff
guile@2.0
gzip
heimdal
historian
homebrew/science/bayestraits
homebrew/science/gnuastro
homebrew/science/oma
homebrew/science/platypusvar
homebrew/science/tamarin-prover
homebrew/science/tensorflow
httpflow
ifuse
immortal
jhipster
jing-trang
juju-wait
jvm-mon
krb5
lapack
latexila
less
libedit
libetpan
libiconv
libpcap
lsof
m4
make
mariadb@10.1
maven@3.3
mono-libgd

Uninstalling hdf5... (193 files, 12.6MB)
==> Installing hdf5 from staticfloat/juliatranslated
==> Downloading https://homebrew.bintray.com/bottles-science/hdf5-1.10.1.el_capitan.bottle.tar.gz
==> Pouring hdf5-1.10.1.el_capitan.bottle.tar.gz
🍺  /Users/edelman/.julia/v0.5/Homebrew/deps/usr/Cellar/hdf5/1.10.1: 203 files, 13MB


Uninstalling imagemagick@6... (1,467 files, 22.3MB)
==> Installing imagemagick@6 from staticfloat/juliatranslated
==> Downloading https://juliabottles.s3.amazonaws.com/imagemagick@6-6.9.7-3.el_capitan.bottle.tar.gz
Already downloaded: /Users/edelman/Library/Caches/Homebrew.jl/imagemagick@6-6.9.7-3.el_capitan.bottle.tar.gz
==> Pouring imagemagick@6-6.9.7-3.el_capitan.bottle.tar.gz
🍺  /Users/edelman/.julia/v0.5/Homebrew/deps/usr/Cellar/imagemagick@6/6.9.7-3: 1,467 files, 22.3MB


Uninstalling libtiff... (248 files, 3.4MB)
==> Installing libtiff from staticfloat/juliatranslated
==> Downloading https://homebrew.bintray.com/bottles/libtiff-4.0.8.el_capitan.bottle.tar.gz
==> Pouring libtiff-4.0.8.el_capitan.bottle.tar.gz
🍺  /Users/edelman/.julia/v0.5/Homebrew/deps/usr/Cellar/libtiff/4.0.8: 245 files, 3.4MB


INFO: Building Blosc
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  118k  100  118k    0     0  86860      0  0:00:01  0:00:01 --:--:--  742k
INFO: Building HDF5
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of BenchmarkTools
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [4]:
using BenchmarkTools  

INFO: Recompiling stale cache file /Users/edelman/.julia/lib/v0.5/JLD.ji for module JLD.


#  1. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [5]:
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [6]:
c_sum(a)

5.001419042481906e6

In [7]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

true

In [8]:
≈  # alias for the `isapprox` function

isapprox (generic function with 2 methods)

In [9]:
?isapprox

search: isapprox



```
isapprox(x, y; rtol::Real=sqrt(eps), atol::Real=0)
```

Inexact equality comparison: `true` if `norm(x-y) <= atol + rtol*max(norm(x), norm(y))`. The default `atol` is zero and the default `rtol` depends on the types of `x` and `y`.

For real or complex floating-point values, `rtol` defaults to `sqrt(eps(typeof(real(x-y))))`. This corresponds to requiring equality of about half of the significand digits. For other types, `rtol` defaults to zero.

`x` and `y` may also be arrays of numbers, in which case `norm` defaults to `vecnorm` but may be changed by passing a `norm::Function` keyword argument. (For numbers, `norm` is the same thing as `abs`.) When `x` and `y` are arrays, if `norm(x-y)` is not finite (i.e. `±Inf` or `NaN`), the comparison falls back to checking whether all elements of `x` and `y` are approximately equal component-wise.

The binary operator `≈` is equivalent to `isapprox` with the default arguments, and `x ≉ y` is equivalent to `!isapprox(x,y)`.


We can now benchmark the C code directly from Julia:

In [9]:
c_bench = @benchmark c_sum($a) 

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     8.051 ms (0.00% GC)
  median time:      8.404 ms (0.00% GC)
  mean time:        8.505 ms (0.00% GC)
  maximum time:     10.338 ms (0.00% GC)
  --------------
  samples:          585
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [10]:
println("C: Fastest time was $(minimum(c_bench.times) / 1e6) msec")

C: Fastest time was 8.051429 msec


In [11]:
d = Dict()  # a "dictionary", i.e. an associative array
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

Dict{Any,Any} with 1 entry:
  "C" => 8.05143

In [12]:
using Plots
gr()

Plots.GRBackend()

In [13]:
t = c_bench.times / 1e6 # times in milliseconds
m, σ = minimum(t), std(t)

histogram(t, bins=500,
    xlim=(m - 0.01, m + σ),
    xlabel="milliseconds", ylabel="count", label="")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 8.1 
 
 
 8.2 
 
 
 8.3 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 12 
 
 
 milliseconds 
 
 
 count

# 2. Python's built in `sum` 

The `PyCall` package provides a Julia interface to Python:

In [14]:
Pkg.add("PyCall")

INFO: No packages to install, update or remove
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of PyCall
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [15]:
using PyCall

In [16]:
# Call a low-level PyCall function to get a Python list, because
# by default PyCall will convert to a NumPy array instead (we benchmark NumPy below):

apy_list = PyCall.array2py(a, 1, 1)

# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [17]:
pysum(a)

5.001419042481906e6

In [18]:
pysum(a) ≈ sum(a)

true

In [19]:
py_list_bench = @benchmark $pysum($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  672 bytes
  allocs estimate:  19
  --------------
  minimum time:     69.597 ms (0.00% GC)
  median time:      72.668 ms (0.00% GC)
  mean time:        73.668 ms (0.00% GC)
  maximum time:     84.844 ms (0.00% GC)
  --------------
  samples:          68
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [20]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any,Any} with 2 entries:
  "C"               => 8.05143
  "Python built-in" => 69.5966

# 3. Python: `numpy` 

## Takes advantage of hardware "SIMD", but only works when it works.

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [21]:
using Conda 
Conda.add("numpy")

Fetching package metadata .........
Solving package specifications: .

Package plan for installation in environment /Users/edelman/.julia/v0.5/Conda/deps/usr:

The following packages will be UPDATED:

    conda: 4.3.14-py27_0 --> 4.3.21-py27_0

conda-4.3.21-p 100% |###############################| Time: 0:00:00   4.80 MB/s


In [22]:
numpy_sum = pyimport("numpy")["sum"]
apy_numpy = PyObject(a) # converts to a numpy array by default

py_numpy_bench = @benchmark $numpy_sum($apy_numpy)

BenchmarkTools.Trial: 
  memory estimate:  960 bytes
  allocs estimate:  25
  --------------
  minimum time:     4.583 ms (0.00% GC)
  median time:      4.884 ms (0.00% GC)
  mean time:        5.058 ms (0.00% GC)
  maximum time:     9.218 ms (0.00% GC)
  --------------
  samples:          980
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [23]:
numpy_sum(apy_list) # python thing

5.00141904248168e6

In [24]:
numpy_sum(apy_list) ≈ sum(a)

true

In [25]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

Dict{Any,Any} with 3 entries:
  "C"               => 8.05143
  "Python numpy"    => 4.5833
  "Python built-in" => 69.5966

# 4. Python, hand-written 

In [26]:
py"""
def py_sum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

sum_py = py"py_sum"

PyObject <function py_sum at 0x339b00b90>

In [27]:
py_hand = @benchmark $sum_py($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  672 bytes
  allocs estimate:  19
  --------------
  minimum time:     1.288 s (0.00% GC)
  median time:      1.290 s (0.00% GC)
  mean time:        1.293 s (0.00% GC)
  maximum time:     1.304 s (0.00% GC)
  --------------
  samples:          4
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [28]:
sum_py(apy_list)

5.001419042481906e6

In [29]:
sum_py(apy_list) ≈ sum(a)

true

In [38]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

Dict{Any,Any} with 6 entries:
  "C"                   => 8.05143
  "Python numpy"        => 4.5833
  "Julia hand-written"  => 8.15885
  "Python hand-written" => 1288.09
  "Python built-in"     => 69.5966
  "Julia built-in"      => 4.48337

# 5. Julia (built-in) 

## Written directly in Julia, not in C!

In [39]:
@which sum(a)

sum(a) at reduce.jl:229

In [40]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.453 ms (0.00% GC)
  median time:      4.832 ms (0.00% GC)
  mean time:        5.005 ms (0.00% GC)
  maximum time:     7.541 ms (0.00% GC)
  --------------
  samples:          990
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [41]:
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any,Any} with 6 entries:
  "C"                   => 8.05143
  "Python numpy"        => 4.5833
  "Julia hand-written"  => 8.15885
  "Python hand-written" => 1288.09
  "Python built-in"     => 69.5966
  "Julia built-in"      => 4.45327

# 6. Julia (hand-written) 

In [42]:
function mysum(A)   
    s = 0.0  # s = zero(eltype(A))
    for a in A
        s += a
    end
    s
end

mysum (generic function with 1 method)

In [43]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     8.118 ms (0.00% GC)
  median time:      8.415 ms (0.00% GC)
  mean time:        8.502 ms (0.00% GC)
  maximum time:     10.760 ms (0.00% GC)
  --------------
  samples:          585
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [44]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any,Any} with 6 entries:
  "C"                   => 8.05143
  "Python numpy"        => 4.5833
  "Julia hand-written"  => 8.11775
  "Python hand-written" => 1288.09
  "Python built-in"     => 69.5966
  "Julia built-in"      => 4.45327

# Julia parallel: `DistributedArrays`

In [1]:
Pkg.add("DistributedArrays")

INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of DistributedArrays
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [2]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [3]:
@everywhere using DistributedArrays

In [6]:
â = distribute(a)  # write â as `a\hat<TAB>`

10000000-element DistributedArrays.DArray{Float64,1,Array{Float64,1}}:
 0.845028 
 0.321898 
 0.0471733
 0.0975601
 0.174604 
 0.480109 
 0.681603 
 0.833004 
 0.48757  
 0.975825 
 0.821781 
 0.339295 
 0.682643 
 ⋮        
 0.446681 
 0.804288 
 0.904222 
 0.116732 
 0.310687 
 0.036794 
 0.0386665
 0.629526 
 0.672523 
 0.980441 
 0.474211 
 0.511631 

In [7]:
â.indexes

4-element Array{Tuple{UnitRange{Int64}},1}:
 (1:2500000,)       
 (2500001:5000000,) 
 (5000001:7500000,) 
 (7500001:10000000,)

In [8]:
â.pids

4-element Array{Int64,1}:
 2
 3
 4
 5

In [9]:
j_parallel = @benchmark sum($â)

LoadError: UndefVarError: @benchmark not defined

In [10]:
d["Julia parallel"] = minimum(j_parallel.times) / 1e6
d

LoadError: UndefVarError: j_parallel not defined

# Summary

In [53]:
for (key, value) in sort(collect(d))
    println(rpad(key, 20, "."), lpad(round(value, 1), 8, "."))
end

C........................8.1
Julia built-in...........4.5
Julia hand-written.......8.1
Julia parallel...........4.0
Python built-in.........69.6
Python hand-written...1288.1
Python numpy.............4.6


In [54]:
for (key, value) in sort(collect(d), by=x->x[2])
    println(rpad(key, 20, "."), lpad(round(value, 2), 10, "."))
end

Julia parallel............3.95
Julia built-in............4.45
Python numpy..............4.58
C.........................8.05
Julia hand-written........8.12
Python built-in...........69.6
Python hand-written....1288.09
